## Connect to your workspace


In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.42.0 to work with ml-lab-eah62qb7osjim


## Prepare data

In [2]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds=ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

workspaceartifactstore - Default = False
workspacefilestore - Default = False
workspaceworkingdirectory - Default = False
workspaceblobstore - Default = True


In [3]:
tab_data_set=Dataset.get_by_name(ws,name="diabetes dataset")
tab_data_set.take(5).to_pandas_dataframe()

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


## Create scripts for pipeline steps

In [4]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'diabetes_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

diabetes_pipeline


Now let's create the first script, which will read data from the diabetes dataset and apply some simple pre-processing to remove any rows with missing data and normalize the numeric features so they're on a similar scale.

The script includes a argument named **--prepped-data**, which references the folder where the resulting data should be saved.

In [17]:
%%writefile $experiment_folder/prep_diabetes.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')

args=parser.parse_args()
save_folder = args.prepped_data


# Get the experiment run context
run= Run.get_context()

# Load the data(passed as an input data)
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# log raw row count
row_count =(len(diabetes))
run.log('raw_rows',row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# End the run
run.complete()


Overwriting diabetes_pipeline/prep_diabetes.py


Now you can create the script for the second step, which will train a model. The script includes a argument named **--training-data**, which references the location where the prepared data was saved by the previous step.

In [34]:
%%writefile $experiment_folder/train_diabetes.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
#parser.add_argument("--model-dir", type=str, dest='model_dir', help='model file')
args = parser.parse_args()
training_data = args.training_data
#outputs=args.model_dir

# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data...")
file_path = os.path.join(training_data,'data.csv')
diabetes = pd.read_csv(file_path)

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train adecision tree model
print('Training a decision tree model...')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'diabetes_model.pkl')
joblib.dump(value=model, filename=model_file)

# Register the model
print('Registering model...')

Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})

'''
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Pipeline'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

'''

run.complete()

Overwriting diabetes_pipeline/train_diabetes.py


## Deploy Script

The deploy step takes the newly trained model and deploys it as a web service endpoint:

**Script file**

In [19]:
%%writefile $experiment_folder/score_diabetes.py
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting diabetes_pipeline/score_diabetes.py


In [20]:
%%writefile $experiment_folder/deploy.py


from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.run import Run
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
import argparse

#model=Model.get_model_path('diabetes_model')

# Define arguments
parser = argparse.ArgumentParser(description='Deploy arg parser')
#parser.add_argument('--test_dir', type=str, help='Directory where testing data is stored')
#parser.add_argument('--model_dir', type=str, help='File storing the evaluation accuracy')
parser.add_argument("--model-dir", type=str, dest='model_dir', help='model file')
#parser.add_argument('--accuracy_dir', type=str, help='File storing the evaluation accuracy')

args = parser.parse_args()

# Get run context
run = Run.get_context()
workspace = run.experiment.workspace

#accuracy_dir = args.accuracy_dir
model_dir = args.model_dir

'''
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
'''
# Configure the scoring environment
service_env = Environment(name='service-env')

# Register environment to re-use later
service_env.register(workspace = workspace)

# Service name
service_name = "diabetes-service"

python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
new_model=True

if new_model:
    inference_config = InferenceConfig(source_directory=experiment_folder,
                                    entry_script="score_diabetes.py",
                                    environment=service_env)

    # Configure the web service container
    deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
    try:
        service=Webservice(workspace,name=service_name)
        if service:
            service.delete()
    except WebserviceException as e:
        print()
    # Deploy the model as a service
    print('Deploying model...')

    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
    service.wait_for_deployment(True)
else:
    service = Webservice(workspace, name=service_name)

print(service.state)
'''
# Output scoring url to file
print(service.scoring_uri)
with open(model_dir + '/scoring_uri.txt', 'w+') as f:
    f.write(service.scoring_uri)
'''

Overwriting diabetes_pipeline/deploy.py


## Prepare a compute environment for the pipeline steps

In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "automl-compute"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


The compute will require a Python environment with the necessary package dependencies installed.

In [22]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting diabetes_pipeline/experiment_env.yml


Now that you have a Conda configuration file, you can create an environment and use it in the run configuration for the pipeline.

In [23]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Register the environment
experiment_env.register(workspace=ws)
registered_env=Environment.get(ws,'experiment_env')

# Create a new runconfig object for the pipemine
pipeline_run_config = RunConfiguration()

# Use the compute you created above
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")


Run configuration created.


## Create and run a pipeline

In [24]:
'''
from azureml.pipeline.core import PipelineData
from azureml.data.data_reference import DataReference

datastore = ws.get_default_datastore()
# Get datastore reference
datastore_reference = DataReference(datastore, mode='mount')


output_dir = PipelineData(
    name='output_dir', 
    pipeline_output_name='outputdir',
    datastore=datastore_reference.datastore,
    output_mode='mount',
    is_directory=True)


model_dir = PipelineData(
    name='model_dir', 
    pipeline_output_name='modeldir',
    datastore=datastore_reference.datastore,
    output_mode='mount',
    is_directory=True)
'''


In [35]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
#from azureml.core.model import Model

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step =PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_diabetes.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_diabetes.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)


#model_dir=Model.get_model_path('diabetes_model')
model_dir = ws.models['diabetes_model']
'''
# Step 3, run the training script
deploy_step = PythonScriptStep(name = "Deploy Model",
                                source_directory = experiment_folder,
                                script_name = "deploy.py",
                                arguments=["--model_dir",model_dir],
                                inputs=[model_dir],
                                #outputs=[output_dir],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)
'''
print("Pipeline steps defined")

Pipeline steps defined


OK, you're ready build the pipeline from the steps you've defined and run it as an experiment.

In [36]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-diabetes-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Prepare Data [ddf4899a][f4dc21bb-d8e7-4917-a389-d4ddc4e9a224], (This step will run and generate new outputs)
Created step Train and Register Model [876dc45d][13a39436-b182-403d-bd6c-62c39c265306], (This step will run and generate new outputs)
Submitted PipelineRun c3aad46f-e2c0-49f5-9854-e2b3ff63c77d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c3aad46f-e2c0-49f5-9854-e2b3ff63c77d?wsid=/subscriptions/1180d7d2-bb0a-4c4a-8011-1a73b31a7318/resourcegroups/cal-3727-34/workspaces/ml-lab-eah62qb7osjim&tid=fd1fbf9f-991a-40b4-ae26-61dfc34421ef
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: c3aad46f-e2c0-49f5-9854-e2b3ff63c77d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c3aad46f-e2c0-49f5-9854-e2b3ff63c77d?wsid=/subscriptions/1180d7d2-bb0a-4c4a-8011-1a73b31a7318/resourcegroups/cal-3727-34/workspaces/ml-lab-eah62qb7osjim&tid=fd1fbf9f-991a-40b4-ae26-61dfc34421ef
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 3a5f6731-6997-4003-ae5f-d5e897f49918
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/3a5f6731-6997-4003-ae5f-d5e897f49918?wsid=/subscriptions/1180d7d2-bb0a-4c4a-8011-1a73b31a7318/resourcegroups/cal-3727-34/workspaces/ml-lab-eah62qb7osjim&tid=fd1fbf9f-991a-40b4-ae26-61dfc34421ef
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished
{'runId': '3a5f6731-6997-4003-ae5f-d5e897f49918', 'target': 'automl-compute', 'status': 'Completed', 'startTimeUtc': '2022-06-22T14:51:10.199222Z', 'endTimeUtc': '2022-06-22T14:52:08.18451Z', 'servic

'Finished'

When the pipeline has finished, you can examine the metrics recorded by it's child runs.

In [37]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

Train and Register Model :
	 Accuracy : 0.8886666666666667
	 AUC : 0.874635554007662
	 ROC : aml://artifactId/ExperimentRun/dcid.a9744dfa-dc2c-4ee2-ab2f-297644385a95/ROC_1655909543.png
Prepare Data :
	 raw_rows : 10000
	 processed_rows : 10000


Assuming the pipeline was successful, a new model should be registered with a Training context tag indicating it was trained in a pipeline. Run the following code to verify this.

In [38]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 2
	 Training context : Pipeline
	 AUC : 0.874635554007662
	 Accuracy : 0.8886666666666667


diabetes_model version: 1
	 Training context : Pipeline
	 AUC : 0.8793571819493506
	 Accuracy : 0.8926666666666667




In [42]:
#%%writefile $experiment_folder/deploy.py


from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace
#from azureml.core.model import Model
from azureml.core.run import Run
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
import argparse

#model=Model.get_model_path('diabetes_model')

'''
# Define arguments
parser = argparse.ArgumentParser(description='Deploy arg parser')
#parser.add_argument('--test_dir', type=str, help='Directory where testing data is stored')
#parser.add_argument('--model_dir', type=str, help='File storing the evaluation accuracy')
parser.add_argument("--model-dir", type=str, dest='model_dir', help='model file')
#parser.add_argument('--accuracy_dir', type=str, help='File storing the evaluation accuracy')

args = parser.parse_args()
'''
# Get run context
run = Run.get_context()
#workspace = run.experiment.workspace

#accuracy_dir = args.accuracy_dir
#model_dir = args.model_dir

'''
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
'''
# Configure the scoring environment
service_env = Environment(name='service-env')

# Register environment to re-use later
service_env.register(workspace = ws)

# Service name
service_name = "diabetes-service"

python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)


# Models
model_dir = ws.models['diabetes_model']

new_model=True

if new_model:
    inference_config = InferenceConfig(source_directory=experiment_folder,
                                    entry_script="score_diabetes.py",
                                    environment=service_env)

    # Configure the web service container
    deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
    try:
        service=Webservice(ws,name=service_name)
        if service:
            service.delete()
    except WebserviceException as e:
        print()
    # Deploy the model as a service
    print('Deploying model...')

    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
    service.wait_for_deployment(True)
else:
    service = Webservice(ws, name=service_name)

print(service.state)
'''
# Output scoring url to file
print(service.scoring_uri)
with open(model_dir + '/scoring_uri.txt', 'w+') as f:
    f.write(service.scoring_uri)
'''


Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-22 14:58:05+00:00 Creating Container Registry if not exists.
2022-06-22 14:58:05+00:00 Registering the environment.
2022-06-22 14:58:06+00:00 Building image..
2022-06-22 15:08:21+00:00 Generating deployment configuration.
2022-06-22 15:08:22+00:00 Submitting deployment to compute..
2022-06-22 15:08:28+00:00 Checking the status of deployment diabetes-service..
2022-06-22 15:09:54+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


"\n# Output scoring url to file\nprint(service.scoring_uri)\nwith open(model_dir + '/scoring_uri.txt', 'w+') as f:\n    f.write(service.scoring_uri)\n"

## Publish the pipeline
After you've created and tested a pipeline, you can publish it as a REST service.

In [43]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="diabetes-training-pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
diabetes-training-pipeline,5f5d0121-72ec-47e4-9098-1d62b5b265e6,Active,REST Endpoint


Note that the published pipeline has an endpoint, which you can see in the Endpoints page (on the Pipeline Endpoints tab) in Azure Machine Learning studio. You can also find its URI as a property of the published pipeline object:

In [44]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westcentralus.api.azureml.ms/pipelines/v1.0/subscriptions/1180d7d2-bb0a-4c4a-8011-1a73b31a7318/resourceGroups/cal-3727-34/providers/Microsoft.MachineLearningServices/workspaces/ml-lab-eah62qb7osjim/PipelineRuns/PipelineSubmit/5f5d0121-72ec-47e4-9098-1d62b5b265e6


##  Call the pipeline endpoint
To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. A real application would require a service principal with which to be authenticated, but to test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

In [45]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Authentication header ready.


Now we're ready to call the REST interface. The pipeline runs asynchronously, so we'll get an identifier back, which we can use to track the pipeline experiment as it runs:

In [46]:
import requests

experiment_name = 'mslearn-diabetes-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

'dd07b553-ea1b-4fb4-b40c-18ffa0cac0bf'

In [47]:
tab_data_set.take(1).to_pandas_dataframe().values.flatten().tolist()[0:9]

[1354778.0, 0.0, 171.0, 80.0, 34.0, 23.0, 43.50972593, 1.213191354, 21.0]

In [56]:
new_data=tab_data_set.take(1).to_pandas_dataframe()[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values.flatten().tolist()
new_data

[0.0, 171.0, 80.0, 34.0, 23.0, 43.50972593, 1.213191354, 21.0]

In [48]:
service.scoring_uri

'http://e9fba7eb-d44b-4b9b-8c1c-3834172405c9.westcentralus.azurecontainer.io/score'

In [57]:
#import urllib.request, urllib.error # urllib.request and urllib.error for Python 3.X
import json
import requests
#from azureml.core.webservice import Webservice

# Iris petal and sepal measurements
x_new =  [new_data]

input_json = json.dumps({"data": x_new})

# Get the URL of the web service
#service = Webservice(workspace=ws, name='iris-classification-service')
endpoint = service.scoring_uri

# Send data to web service
#body = str.encode(json.dumps(rawdata))

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )


Patient [0.0, 171.0, 80.0, 34.0, 23.0, 43.50972593, 1.213191354, 21.0] diabetic
